In [144]:
#librerias
import pandas as pd
from geopy.geocoders import Nominatim
import requests


In [145]:
#Test API
# URL base de la API
url_base = 'https://api.openchargemap.io/v3'

# Parámetros de la consulta
parameters = {
    'key': 'e5946eb4-a02b-44e7-8198-d347ee82b1b3',  
    'output': 'json',  
    'DistanceUnit': 'kilometers',
    'maxresults': 50 ,
    'latitude': '16.753801',
    'longitude': '-93.115959'
}

# Realizar la solicitud GET a la API
response = requests.get(f'{url_base}/poi', params=parameters)

# Verificar el código de estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa
    data = response.json()

else:
    # La solicitud falló
    print(f'Error al consultar la API. Código de estado: {response.status_code}')



In [149]:
#testeamos la paqueteria
geolocator = Nominatim(user_agent="seg_auto_2")
location = geolocator.geocode("TLAJOMULCO DE ZUñIGA")
print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

Tlajomulco de Zuñiga, Jalisco, México
(20.48187375, -103.40050966382874)
{'place_id': 298813959, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 5606517, 'boundingbox': ['20.3596958', '20.6041925', '-103.5966497', '-103.1980384'], 'lat': '20.48187375', 'lon': '-103.40050966382874', 'display_name': 'Tlajomulco de Zuñiga, Jalisco, México', 'class': 'boundary', 'type': 'administrative', 'importance': 0.7500100000000001, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}


In [171]:
#funciones

#contador de estaciones
def conteo(json_respuesta):
    cerca = 1
    contador=0
    try:
        while cerca > 0:

            if json_respuesta[contador]['AddressInfo']['Distance']<10:
                contador += 1
            else:
                cerca=0
    except Exception as e:
        # Manejo de la excepción
        print(f"Se produjo un error: {str(e)}" )
        
    return contador 

# consulta de api

def open_charge_apo(lat,long):
    #seteamos los parametros
    parameters['latitude']=lat #catalogo['latitude'][0]
    parameters['longitude']=long#catalogo['longitude'][0]
    #Consultamos la api
    response = requests.get(f'{url_base}/poi', params=parameters)
    try:
        if response.status_code == 200:
            data = response.json()
        else:
        # La solicitud falló
            print(f'Error al consultar la API. Código de estado: {response.status_code}')
    except Exception as e:
        # Manejo de la excepción
        print(f"Se produjo un error: {str(e)}" )
    return data

In [147]:
#carga de insumo
auto_df = pd.read_csv('auto_coloca_2022_v2_clean.csv')

In [148]:
#generaremos un catalogo para reducir la consulta a la api
catalogo=auto_df["town_name_cte"].drop_duplicates().reset_index()
#Insertamos la letral "ñ" para obtener la información de forma correcta
catalogo=catalogo["town_name_cte"].replace('Ã‘','ñ', regex=True)

In [150]:
#Transformamos de serie a dataframe para poder agregar columnas a discreción
catalogo = catalogo.to_frame().reset_index(drop=True)

#agregamos las columnas de interes
catalogo["latitude"]=""
catalogo["longitude"]=""
catalogo["num_estaciones"]=""

In [151]:
#llenamos el catalogo con latitud y longitud
geolocator = Nominatim(user_agent="seg_auto")
len_df=len(catalogo)


for i in range(len_df):
    try:
        location = geolocator.geocode(catalogo["town_name_cte"][i])
        catalogo["latitude"][i]=location.latitude
        catalogo["longitude"][i]=location.longitude
    except Exception as e:
        # Manejo de la excepción
        print(f"Se produjo un error: {str(e)}" )
        continue




KeyboardInterrupt: 

In [ ]:
#revision exploratorio de las fallas
with_out_lat = catalogo['latitude'] == ""
catalogo_with_out_lat = catalogo[with_out_lat]
catalogo_with_out_lat.head()



,town_name_cte,latitude,longitude
305,DOLORES HIDALGO CUNA DE LA IND,,
391,IXTACUIXTLA DE MARIANO MATAMOR,,
435,NANCHITAL DE LAZARO CARDENAS D,,
508,ALTO LUCERO DE GUTIERREZ BARRI,,
584,SANTIAGO TULANTEPEC DE LUGO GU,,


In [155]:
#guardamos para no repetir por si se cierra el entorno
catalogo.to_csv('catalogo.csv')

In [158]:
catalogo=pd.read_csv('catalogo.csv')

In [173]:

for i in range(len(catalogo)):
    
    catalogo['num_estaciones'][i]=conteo(open_charge_apo(catalogo['latitude'][i],catalogo['longitude'][i]))



C:\Users\León\AppData\Local\Temp\ipykernel_11100\2908209441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catalogo['num_estaciones'][i]=conteo(open_charge_apo(catalogo['latitude'][i],catalogo['longitude'][i]))


Se produjo un error: list index out of range
Se produjo un error: list index out of range


KeyboardInterrupt: 

In [174]:
catalogo

,Unnamed: 0.1,Unnamed: 0,town_name_cte,latitude,longitude,num_estaciones
0,0,0,GUADALUPE VICTORIA,19.307352,-97.403055,0.0
1,1,1,IXTAPALUCA,19.317098,-98.905014,1.0
2,2,2,CARDENAS,42.374363,-2.767099,1.0
3,3,3,ZACAPOAXTLA,19.840705,-97.579300,0.0
4,4,4,TUXTLA GUTIERREZ,16.753801,-93.115959,2.0
...,...,...,...,...,...,...
1686,1686,1686,SAN DIEGO DE ALEJANDRIA,20.955997,-102.026187,NaN
1687,1687,1687,PETLALCINGO,18.062810,-97.909793,NaN
1688,1688,1688,SANTA MARIA ALOTEPEC,17.084869,-95.830359,NaN
1689,1689,1689,SANTA LUCIA MONTEVERDE,16.955329,-97.721327,NaN


In [178]:
catalogo=catalogo[['town_name_cte','num_estaciones']]

In [179]:
auto_coloca_df_geo = auto_df.merge(catalogo, how='left', on='town_name_cte')